In [1]:
import time
import warnings
import pandas as pd
import numpy as np
import os
import psutil
from sklearn.base import clone
from sklearn.datasets import make_classification
from memory_profiler import memory_usage

# --- Your other imports ---
from skrebate import ReliefF, SURF, SURFstar, MultiSURF as SkrebateMultiSURF, MultiSURFstar
from src.fast_select.ReliefF import ReliefF as FastReliefF
from src.fast_select.SURF import SURF as FastSURF
from src.fast_select.MultiSURF import MultiSURF as FastMultiSURF

# --- GPU Detection ---
try:
    from numba import cuda
    GPU_AVAILABLE = cuda.is_available()
except (ImportError, cuda.cudadrv.error.CudaSupportError):
    GPU_AVAILABLE = False

# --- Benchmark Configuration ---
P_DOMINANT_SCENARIOS = {
    "name": "p >> n (Features Dominant)",
    "fixed_param": "n_samples", "fixed_value": 500,
    "varied_param": "n_features", "varied_range": [100, 200, 300, 400, 500]
}
N_DOMINANT_SCENARIOS = {
    "name": "n >> p (Samples Dominant)",
    "fixed_param": "n_features", "fixed_value": 100,
    "varied_param": "n_samples", "varied_range": [500, 1000, 1500, 2000, 2500]
}
N_FEATURES_TO_SELECT = 10
N_REPEATS = 3 # Increase repeats for more stable results

# --- Estimators to Test ---
estimators = {
    # skrebate estimators
    "skrebate.ReliefF": ReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, n_jobs=-1),
    "skrebate.SURF": SURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    "skrebate.MultiSURF": SkrebateMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    # fast-select CPU estimators
    "fast_select.ReliefF (CPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, backend='cpu', n_jobs=-1),
    "fast_select.SURF (CPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    "fast_select.MultiSURF (CPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='cpu', n_jobs=-1),
}
if GPU_AVAILABLE:
    print("NVIDIA GPU detected. Including GPU benchmarks.")
    estimators.update({
        "fast_select.ReliefF (GPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, backend='gpu'),
        "fast_select.SURF (GPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='gpu'),
        "fast_select.MultiSURF (GPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='gpu'),
    })
else:
    print("No NVIDIA GPU detected. Skipping GPU benchmarks.")

# --- CORRECTED BENCHMARK FUNCTION ---
def run_single_benchmark(estimator, X, y):
    """
    Measures runtime and peak memory usage of a single estimator fit.
    This version performs only ONE execution and correctly measures GPU memory.
    """
    is_gpu_estimator = hasattr(estimator, 'backend') and estimator.backend == 'gpu'
    
    # Use a lambda to wrap the fit call
    fit_func = lambda: estimator.fit(X, y)

    # --- Memory Measurement ---
    peak_mem_mb = 0
    if is_gpu_estimator:
        # For GPU, we measure VRAM usage directly.
        # This requires the fit function to be run inside the context.
        ctx = cuda.current_context()
        start_mem = ctx.get_memory_info().free
        fit_func() # Run the function
        end_mem = ctx.get_memory_info().free
        # Peak memory is the reduction in free memory.
        peak_mem_mb = (start_mem - end_mem) / (1024**2)
    else:
        # For CPU, memory_profiler works perfectly.
        mem_samples = memory_usage(fit_func, interval=0.1)
        peak_mem_mb = max(mem_samples)

    # --- Runtime Measurement ---
    # Since the function has already been run once for memory profiling,
    # we time a second run to get a pure execution time without JIT overhead.
    # This is now a consistent measurement.
    start_time = time.perf_counter()
    fit_func()
    end_time = time.perf_counter()
    runtime = end_time - start_time
    
    return runtime, peak_mem_mb

def warmup_jit_compilers(estimators_dict):
    """Performs a 'warm-up' run on a small dataset to compile JIT functions."""
    print("\n--- Warming up JIT compilers ---")
    X_warmup, y_warmup = make_classification(n_samples=20, n_features=20, random_state=42)
    for name, estimator in estimators_dict.items():
        # More robust check for our custom estimators
        if "fast_select" in name:
            print(f"  Warming up {name}...")
            try:
                # Use a fresh clone for warmup
                clone(estimator).fit(X_warmup, y_warmup)
            except Exception as e:
                warnings.warn(f"  > Warm-up FAILED for {name}. Reason: {type(e).__name__}: {e}")
    print("--- Warm-up complete ---")

def main():
    """Main function to run all benchmark scenarios."""
    results = []
    warmup_jit_compilers(estimators)

    scenarios = [P_DOMINANT_SCENARIOS, N_DOMINANT_SCENARIOS]
    
    for scenario_params in scenarios:
        scenario_name = scenario_params["name"]
        print(f"\n--- Running Scenario: {scenario_name} ---")
        
        fixed_param = scenario_params["fixed_param"]
        varied_param = scenario_params["varied_param"]
        
        for varied_value in scenario_params["varied_range"]:
            # Set up dataset dimensions for this run
            if fixed_param == "n_samples":
                n_samples = scenario_params["fixed_value"]
                n_features = varied_value
            else:
                n_samples = varied_value
                n_features = scenario_params["fixed_value"]
                
            print(f"\nGenerating data: {n_samples} samples, {n_features} features")
            X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=20, n_redundant=50, random_state=42)

            for name, base_estimator in estimators.items():
                for i in range(N_REPEATS):
                    print(f"  Benchmarking {name} (Run {i+1}/{N_REPEATS})...")
                    try:
                        estimator = clone(base_estimator)
                        runtime, peak_mem = run_single_benchmark(estimator, X, y)
                        results.append({
                            "scenario": scenario_name, "algorithm": name,
                            "n_samples": n_samples, "n_features": n_features,
                            "runtime_sec": runtime, "peak_memory_mb": peak_mem
                        })
                    except Exception as e:
                        warnings.warn(f"  > FAILED: {name} on {n_samples}x{n_features}. Reason: {type(e).__name__}: {e}", UserWarning)

    # --- Save results to CSV ---
    df = pd.DataFrame(results)
    output_file = "benchmark_results_with_memory.csv"
    df.to_csv(output_file, index=False)
    print(f"\nBenchmarking complete. Results saved to '{output_file}'")

if __name__ == "__main__":
    main()

NVIDIA GPU detected. Including GPU benchmarks.

--- Warming up JIT compilers ---
  Warming up fast_select.ReliefF (CPU)...


/home/galynch/snap/snapd-desktop-integration/253/Desktop/FastSelect/src/fast_select/ReliefF.py:360: UserWarning: n_neighbors (10) is greater than or equal to the smallest class size (10).
  warnings.warn(
/home/galynch/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


  Warming up fast_select.SURF (CPU)...
  Warming up fast_select.MultiSURF (CPU)...
  Warming up fast_select.ReliefF (GPU)...


/home/galynch/snap/snapd-desktop-integration/253/Desktop/FastSelect/src/fast_select/ReliefF.py:360: UserWarning: n_neighbors (10) is greater than or equal to the smallest class size (10).
  warnings.warn(


  Warming up fast_select.SURF (GPU)...
  Warming up fast_select.MultiSURF (GPU)...
--- Warm-up complete ---

--- Running Scenario: p >> n (Features Dominant) ---

Generating data: 500 samples, 100 features
  Benchmarking skrebate.ReliefF (Run 1/3)...
  Benchmarking skrebate.ReliefF (Run 2/3)...
  Benchmarking skrebate.ReliefF (Run 3/3)...
  Benchmarking skrebate.SURF (Run 1/3)...
  Benchmarking skrebate.SURF (Run 2/3)...
  Benchmarking skrebate.SURF (Run 3/3)...
  Benchmarking skrebate.MultiSURF (Run 1/3)...
  Benchmarking skrebate.MultiSURF (Run 2/3)...
  Benchmarking skrebate.MultiSURF (Run 3/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 1/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 2/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 3/3)...
  Benchmarking fast_select.SURF (CPU) (Run 1/3)...
  Benchmarking fast_select.SURF (CPU) (Run 2/3)...
  Benchmarking fast_select.SURF (CPU) (Run 3/3)...
  Benchmarking fast_select.MultiSURF (CPU) (Run 1/3)...
  Benchmarking fast_s

  Benchmarking fast_select.ReliefF (GPU) (Run 1/3)...
  Benchmarking fast_select.ReliefF (GPU) (Run 2/3)...
  Benchmarking fast_select.ReliefF (GPU) (Run 3/3)...
  Benchmarking fast_select.SURF (GPU) (Run 1/3)...
  Benchmarking fast_select.SURF (GPU) (Run 2/3)...
  Benchmarking fast_select.SURF (GPU) (Run 3/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 1/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 2/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 3/3)...

Generating data: 1000 samples, 100 features
  Benchmarking skrebate.ReliefF (Run 1/3)...
  Benchmarking skrebate.ReliefF (Run 2/3)...
  Benchmarking skrebate.ReliefF (Run 3/3)...
  Benchmarking skrebate.SURF (Run 1/3)...
  Benchmarking skrebate.SURF (Run 2/3)...
  Benchmarking skrebate.SURF (Run 3/3)...
  Benchmarking skrebate.MultiSURF (Run 1/3)...
  Benchmarking skrebate.MultiSURF (Run 2/3)...
  Benchmarking skrebate.MultiSURF (Run 3/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 1/3)...
  Benchmarking f

In [9]:
!git commit -a -m "fixed implementations to be gpu and cpu consistent"
!git push

On branch development
Your branch is up to date with 'origin/development'.

nothing to commit, working tree clean
Everything up-to-date


In [8]:
!git status


On branch development
Your branch is up to date with 'origin/development'.

nothing to commit, working tree clean


In [7]:
!git add .

In [6]:
!git checkout development

Branch 'development' set up to track remote branch 'development' from 'origin'.
Switched to a new branch 'development'


In [ ]:
from skrebate import MultiSURFstar as SKMultiSURF
from skrebate import SURFstar as SURFSK
from skrebate import ReliefF as ReliefFSK
import numpy as np
def generate_large_robust_dataset(n_samples=1000, seed=29):
    np.random.seed(seed)
    # Create balanced binary classes
    n0 = n_samples // 2
    n1 = n_samples - n0
    y = np.array([0] * n0 + [1] * n1, dtype=np.int32)

    # Feature 0: Highly relevant continuous feature.
    # Class 0 samples come from N(1, 1), class 1 samples come from N(10, 1).
    f0_class0 = np.random.normal(loc=1.0, scale=1.0, size=n0)
    f0_class1 = np.random.normal(loc=10.0, scale=1.0, size=n1)
    f0 = np.concatenate([f0_class0, f0_class1])

    # Feature 1: Irrelevant noise feature.
    # Values drawn from a standard normal distribution regardless of class.
    f1 = np.random.normal(loc=0.0, scale=1.0, size=n_samples)

    # Feature 2: Relevant discrete feature.
    # Class 0 predominantly gets value 10 but with a small chance for 20,
    # and class 1 predominantly gets value 20 but with a small chance for 10.
    f2_class0 = np.random.choice([10, 20], size=n0, p=[0.9, 0.1])
    f2_class1 = np.random.choice([20, 10], size=n1, p=[0.9, 0.1])
    f2 = np.concatenate([f2_class0, f2_class1])

    # Feature 3: Irrelevant constant feature.
    f3 = np.full((n_samples,), 3.0)

    # Assemble features into one array.
    X = np.column_stack([f0, f1, f2, f3]).astype(np.float32)

    # Shuffle the dataset (so the classes are randomly interleaved)
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    return X, y

X,y = generate_large_robust_dataset()
model = SKMultiSURF()
model.fit(X, y)
print(model.feature_importances_)
model = SURFSK()
model.fit(X, y)
print(model.feature_importances_)
model = ReliefFSK()
model.fit(X, y)
print(model.feature_importances_)

In [ ]:
from src.fast_select import MultiSURF, ReliefF, SURF
import numpy as np
def generate_large_robust_dataset(n_samples=10000, seed=59):
    np.random.seed(seed)
    # Create balanced binary classes
    n0 = n_samples // 2
    n1 = n_samples - n0
    y = np.array([0] * n0 + [1] * n1, dtype=np.int32)

    # Feature 0: Highly relevant continuous feature.
    # Class 0 samples come from N(1, 1), class 1 samples come from N(10, 1).
    f0_class0 = np.random.normal(loc=1.0, scale=1.0, size=n0)
    f0_class1 = np.random.normal(loc=10.0, scale=1.0, size=n1)
    f0 = np.concatenate([f0_class0, f0_class1])

    # Feature 1: Irrelevant noise feature.
    # Values drawn from a standard normal distribution regardless of class.
    f1 = np.random.normal(loc=0.0, scale=1.0, size=n_samples)

    # Feature 2: Relevant discrete feature.
    # Class 0 predominantly gets value 10 but with a small chance for 20,
    # and class 1 predominantly gets value 20 but with a small chance for 10.
    f2_class0 = np.random.choice([10, 20], size=n0, p=[0.9, 0.1])
    f2_class1 = np.random.choice([20, 10], size=n1, p=[0.9, 0.1])
    f2 = np.concatenate([f2_class0, f2_class1])

    # Feature 3: Irrelevant constant feature.
    f3 = np.full((n_samples,), 3.0)

    # Assemble features into one array.
    X = np.column_stack([f0, f1, f2, f3]).astype(np.float32)

    # Shuffle the dataset (so the classes are randomly interleaved)
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    return X, y

X,y = generate_large_robust_dataset()
model = MultiSURF(discrete_limit=4, use_star=True)
model.fit(X, y)
print(model.feature_importances_)
X,y = generate_large_robust_dataset()
model = MultiSURF(discrete_limit=4, backend='cpu', use_star=True)
model.fit(X, y)
print(model.feature_importances_)
model = ReliefF(discrete_limit=4)
model.fit(X, y)
print(model.feature_importances_)
model = ReliefF(discrete_limit=4, backend='cpu')
model.fit(X, y)
print(model.feature_importances_)
model = SURF(discrete_limit=4, use_star=True)
model.fit(X, y)
print(model.feature_importances_)
model = SURF(discrete_limit=4, backend='cpu', use_star=True)
model.fit(X, y)
print(model.feature_importances_)

In [4]:
# plot_benchmarks.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

def plot_scenario(df, scenario_name, x_axis, y_axis, y_label, title, filename, use_log_scale=True):
    """
    Generic helper function to generate and save a plot for a given scenario.
    
    Args:
        df (pd.DataFrame): The full results dataframe.
        scenario_name (str): The name of the scenario to filter for (e.g., 'p >> n').
        x_axis (str): The column name for the x-axis (e.g., 'n_features').
        y_axis (str): The column name for the y-axis (e.g., 'runtime_sec').
        y_label (str): The descriptive label for the y-axis.
        title (str): The main title for the plot.
        filename (str): The output filename for the saved plot.
        use_log_scale (bool): Whether to use a logarithmic scale for the y-axis.
    """
    # Filter the DataFrame for the specific scenario
    scenario_df = df[df['scenario'] == scenario_name].copy()

    # Create a new figure and axes for the plot
    plt.figure(figsize=(12, 8))
    
    # Use seaborn for a clean, publication-quality line plot
    sns.lineplot(
        data=scenario_df,
        x=x_axis,
        y=y_axis,
        hue='algorithm',
        marker='o',
        linewidth=2.5,
        errorbar='sd' # 'ci' is deprecated; 'errorbar' is the new standard
    )
    
    # Set plot properties
    plt.title(title, fontsize=18, fontweight='bold', pad=20)
    plt.xlabel(x_axis.replace('_', ' ').title(), fontsize=14)
    plt.ylabel(y_label, fontsize=14)
    
    if use_log_scale:
        plt.yscale('log')
    
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.legend(title='Algorithm', fontsize=11, title_fontsize=13)
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Plot saved to '{filename}'")
    plt.close() # Close the figure to free up memory

def main():
    """Main function to load results and generate all plots."""
    input_file = "benchmark_results_with_memory.csv"
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"Error: '{input_file}' not found.", file=sys.stderr)
        print("Please run the updated benchmark script first.", file=sys.stderr)
        sys.exit(1)

    # Validate that the necessary columns exist
    required_cols = ['scenario', 'algorithm', 'n_samples', 'n_features', 'runtime_sec', 'peak_memory_mb']
    if not all(col in df.columns for col in required_cols):
        print(f"Error: The CSV file '{input_file}' is missing required columns.", file=sys.stderr)
        print(f"Expected columns: {required_cols}", file=sys.stderr)
        print(f"Found columns: {list(df.columns)}", file=sys.stderr)
        sys.exit(1)

    # Use a professional plot style
    sns.set_theme(style="whitegrid")

    # --- Runtime Plots ---
    print("\n--- Generating Runtime Plots ---")
    plot_scenario(
        df=df,
        scenario_name='p >> n (Features Dominant)',
        x_axis='n_features',
        y_axis='runtime_sec',
        y_label="Runtime (seconds, log scale)",
        title='Benchmark: Runtime vs. Number of Features (p >> n)\n(n_samples fixed)',
        filename='benchmark_p_dominant_runtime.png',
        use_log_scale=True
    )
    plot_scenario(
        df=df,
        scenario_name='n >> p (Samples Dominant)',
        x_axis='n_samples',
        y_axis='runtime_sec',
        y_label="Runtime (seconds, log scale)",
        title='Benchmark: Runtime vs. Number of Samples (n >> p)\n(n_features fixed)',
        filename='benchmark_n_dominant_runtime.png',
        use_log_scale=True
    )

    # --- Memory Usage Plots ---
    print("\n--- Generating Memory Usage Plots ---")
    plot_scenario(
        df=df,
        scenario_name='p >> n (Features Dominant)',
        x_axis='n_features',
        y_axis='peak_memory_mb',
        y_label="Peak Memory Usage (MB, log scale)",
        title='Benchmark: Memory vs. Number of Features (p >> n)\n(n_samples fixed)',
        filename='benchmark_p_dominant_memory.png',
        use_log_scale=True # Memory can also vary greatly, log scale is often useful
    )
    plot_scenario(
        df=df,
        scenario_name='n >> p (Samples Dominant)',
        x_axis='n_samples',
        y_axis='peak_memory_mb',
        y_label="Peak Memory Usage (MB, log scale)",
        title='Benchmark: Memory vs. Number of Samples (n >> p)\n(n_features fixed)',
        filename='benchmark_n_dominant_memory.png',
        use_log_scale=True
    )

if __name__ == "__main__":
    main()


--- Generating Runtime Plots ---
Plot saved to 'benchmark_p_dominant_runtime.png'
Plot saved to 'benchmark_n_dominant_runtime.png'

--- Generating Memory Usage Plots ---
Plot saved to 'benchmark_p_dominant_memory.png'
Plot saved to 'benchmark_n_dominant_memory.png'
